In [4]:
import inspect

In [1]:
# 验证format
text_unformat = '{a}, {b}, {c}, {d}, {e}, {f}'
text_data = {'a': 'a', 'b': 'b', 'c': 'c'}
print(text_unformat.format(**text_data))

KeyError: 'd'

In [16]:
temp_doc = """
获取函数的描述内容
    
Args:
    a (int): param a
    b (str): param b
    c (list, optional): param c. Defaults to None.

Returns:
    d: result d
"""
def agent_fun(a: int, b: str, c: list = None):
    """
    获取函数的描述内容
    
    Args:
        a (int): param a
        b (str): param b
        c (list, optional): param c. Defaults to None.
    Returns:
        d: result d
    """
    print(f'{a}, {b}, {c}')

def agent_desc(agent_name, fun):
    fun_doc = inspect.getdoc(fun)
    fun_args = inspect.getfullargspec(fun)
    # print(f'doc:{fun_doc}')
    arg_list = fun_args.args
    print(f'args:{arg_list}')
    doc_lines = fun_doc.split('\n')
    func_desc = ''
    for line in doc_lines: 
        if line.strip() == '':
            continue
        if line.strip() == 'Args:':
            break
        func_desc += f'{line}.'
    func_args_desc, start = [], False
    match_arg = []
    for line in doc_lines:
        line = line.strip()
        if line == 'Args:':
            start = True
        elif 'Return' in line:
            break
        elif start:
            arg_left = line.split(':')[0]
            if '(' not in arg_left or ')' not in arg_left:
                raise ValueError(f'agent: {agent_name} 的doc注释格式化不正确\n{temp_doc}')
            arg_name = arg_left.split('(')[0].strip()
            print(arg_name, arg_left)
            if arg_name not in arg_list:
                raise ValueError(f'agent: {agent_name} 的doc注释描述中的参数:{arg_name}和实际参数未匹配')
            func_args_desc.append(line)
            for i, arg in enumerate(arg_list):
                if arg == arg_name:
                    match_arg.append(i)
    if len(match_arg) != len(arg_list):
        raise ValueError(f'agent: {agent_name} 的doc注释中未将所有的参数进行描述完全')                    
    print(f'func_desc:{func_desc}')
    print(f'func_args_desc:{func_args_desc}')

agent_desc('agent_fun', agent_fun)

args:['a', 'b', 'c']
a a (int)
b b (str)
c c (list, optional)
func_desc:获取函数的描述内容.
func_args_desc:['a (int): param a', 'b (str): param b', 'c (list, optional): param c. Defaults to None.']


In [ ]:
dir

In [8]:
import subprocess
# command = ['C:/Windows/System32/WindowsPowerShell/v1.0/powershell.exe', 'dir']
# command = ['powershell.exe', 'dir']
command = 'powershell.exe C:\Windows\System32\calc.exe'
result = subprocess.run(command, check=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
print(result)


CompletedProcess(args='powershell.exe C:\\Windows\\System32\\calc.exe', returncode=0, stdout='', stderr='')


In [2]:
pip install mistune

Note: you may need to restart the kernel to use updated packages.


In [15]:
from pprint import pprint

In [23]:
import json
from pprint import pprint

f = open('../doc/create_project_answer.md', 'r')
titles = ['用户需求', '验证项目']
title_idx = 0
contents = {}
content, find_title, start_content = "", -1, False
# 获取```类型，方便后续进行转化，目前主要是json需要转化，其他占不需要
content_type = 'shell'

# 判定当前内容是否在三个引号中，表示是python code
tri_quota = False
for line in f.readlines():
    # print(f'line:{line}')
    if title_idx < len(titles) and titles[title_idx] in line:
        if find_title != -1:
            if content_type == 'json':
                contents[titles[title_idx-1]] = json.loads(content)
            else:
                contents[titles[title_idx-1]] = content
            content = ""
        find_title = title_idx
        title_idx = title_idx + 1
    elif find_title >= 0 and '```' in line:
        # 判定是否在输出中，prompt要求，输出需要执行的内容必须在```中，而且需要尽量保证每一个```中的内容是原子性的
        # 比如: 一个```json 中尽量不要存在两个json,后续会进行优化判定，当代码是不支持的
        new_line = line.replace('\n', '').strip()
        if len(new_line) > 3:
            content_type = str(new_line[3:])
        start_content = not start_content
    elif start_content:
        # python代码中的"""转义json会报错
        # python代码中的换行也会报错，都需要转义
        if '"""' in line:
            line = line.replace('"""', '"')
            tri_quota = not tri_quota
        if tri_quota:
            line = line.replace('\n', '@@')
        content += line
if content != '':
    # 最后一个需要放到字典中去
    if content_type == 'json':
        contents[titles[title_idx-1]] = json.loads(content)
    else:
        contents[titles[title_idx-1]] = content
contents_str=json.dumps(contents,ensure_ascii=False, indent=4)
print(contents_str)

{
    "用户需求": [
        {
            "desc": "创建项目目录test",
            "create_dir": {
                "args": {
                    "dir_path": "./test"
                }
            }
        },
        {
            "desc": "在test目录下创建并写入包含hello world打印功能的test.py文件",
            "write_file": {
                "args": {
                    "file_path": "./test/test.py",
                    "file_content": "@@                    def hello_world():@@                        print('hello world')@@@@                    if __name__ == '__main__':@@                        hello_world()@@                "
                }
            }
        }
    ],
    "验证项目": [
        {
            "desc": "在本地环境中直接运行test.py文件以验证输出hello world",
            "windows_shell_agent": {
                "args": {
                    "command": "python ./test/test.py"
                }
            }
        }
    ]
}


In [22]:
# 将python代码部分格式化，方便后续写入python
def format_python(code: str):
    cs = code.split('@@')
    empty_len = 0
    for c in cs:
        if len(c) != 0:
            empty_len = len(c) - len(c.lstrip())
            break
    new_code = ""
    for c in cs:
        if len(c) > empty_len:
            new_code += str(c[empty_len:])+'\n'
        else:
            new_code += c+'\n'
    return new_code

print(format_python(contents['用户需求'][1]['write_file']['args']['file_content']))


def hello_world():
    print('hello world')

if __name__ == '__main__':
    hello_world()
                



In [15]:
import time
import pyperclip
from DrissionPage import ChromiumPage

question = "用python写一个Hello World"

# 请求千问大模型，并通过复制按钮获取返回的内容
page = ChromiumPage(6666)
# page.get('https://tongyi.aliyun.com/qianwen/')
new_page = page.get_tab(page.latest_tab).page

textarea_ele = new_page.ele('tag:textarea@@class:ant-input')
textarea_ele.input(question)
new_page.ele('css:div.chatBtn--RFpkrgo_ > span.anticon').click()

time.sleep(0.3)
answers = new_page.eles('css:div.answerItem--U4_Uv3iw')
last_answer = ''
answer_ele = answers[-1]
# flex_ele = new_page.ele('tag:div@@class=flexCon')
parse = answer_ele.ele('css:div.rightArea--waHL0DVo > div:nth-child(3)')
print(parse)
while parse is None:
    print('等待5s')
    time.sleep(5)
    parse = answer_ele.ele('css:div.rightArea--waHL0DVo > div:nth-child(3)')
parse.click()
time.sleep(1)
answer = pyperclip.paste()
print(f'千问模型问答: question:{question} answer:{answer}')
# #chat-content > div.flex.mt-4 > div.content--oYlo131R > div.react-scroll-to-bottom--css-aywev-79elbk.w-full.flex-1.overflow-auto.scrollWrap--Cc0w8zsU > div > div.answerItem--U4_Uv3iw > div.containerWrap--lFLVsVCe > div.tools--IJfoLgka > div.rightArea--waHL0DVo > div:nth-child(3)

<ChromiumElement div class='btn--Bw0FbWYV'>
千问模型问答: question:用python写一个Hello World answer:```python
print("Hello World")
```


In [29]:
import time
import pyperclip
from DrissionPage import ChromiumPage

# 请求千问大模型，并通过复制按钮获取返回的内容
page = ChromiumPage(6666)
# page.get('https://tongyi.aliyun.com/qianwen/')
new_page = page.get_tab(page.latest_tab).page

answers = new_page.eles('css:div.answerItem--U4_Uv3iw')
last_answer = ''
answer_ele = answers[-1]
print(answer_ele)

<ChromiumElement div class='answerItem--U4_Uv3iw'>


In [30]:
parse = answer_ele.ele('css:div.rightArea--waHL0DVo > div:nth-child(3)')
print(parse)
parse.click()
time.sleep(1)
answer = pyperclip.paste()
print(answer)

<ChromiumElement div class='btn--Bw0FbWYV'>
### 项目结构
```
test/
├── test.py
```

### 用户需求拆解如下:

```json
[
    {
        "desc": "创建项目目录并写入test.py文件",
        "create_dir": {
            "dir_path": "test",
            "file_path": "test/test.py",
            "file_content": """def print_hello_world():
    print('Hello World')
"""
        }
    }
]
```

### 验证项目效果拆解如下:

```json
[
    {
        "desc": "执行test.py文件中的print_hello_world函数",
        "run_python": {
            "code": """
import sys
sys.path.append('test')
from test import print_hello_world

print_hello_world()
""",
            "return_keys": ["output"]
        }
    }
]
```

### 理解与优化策略：

1. **对自己的能力与行为的匹配进行review**：在本次任务中，我选择了`create_dir`和`run_python`这两个Agent，因为它们分别能高效地创建含有指定文件的目录结构以及验证Python脚本的功能，这与用户需求完美匹配。直接在`create_dir`的一步中写入文件内容，避免了额外的文件写入操作，提高了效率。

2. **大局观与自我反思**：从整体任务流程来看，先构建测试项目结构并写入代码，随后验证其正确性，这样的顺序符合软件开发的逻辑流程。考虑到可能存在的后续扩展需求，保持项目结构的简洁性和代码的可读性至关重要。

3. **长期记忆对决策动作进行优化**：基于历史经验，理解到直接操作文件系统和执行Python脚本是处理此类任务的最直接方式。未来

In [37]:
import json
# 获取千问的md，并解析
def _parse_md(markdown: str, titles: list):
    """
    解析markdown中的内容
    Args:
        markdown (str): markdown内容
        titles (list): 需要抽取的内容标题
    """
    if titles is None or len(titles) == 0:
        return {}
    title_idx = 0
    contents = {}
    content, find_title, start_content = "", -1, False
    # 获取```类型，方便后续进行转化，目前主要是json需要转化，其他占不需要
    content_type = 'shell'

    # 判定当前内容是否在三个引号中，表示是python code
    tri_quota = False
    loop = 0
    for line in markdown.split('\n'):
        # print(loop, line)
        loop+=1
        if title_idx < len(titles) and titles[title_idx] in line:
            if find_title != -1:
                if content_type == 'json':
                    # print(f'content:{content}')
                    contents[titles[title_idx-1]] = json.loads(content)
                else:
                    contents[titles[title_idx-1]] = content
                content = ""
            find_title = title_idx
            title_idx = title_idx + 1
        elif find_title >= 0 and '```' in line:
            # 判定是否在输出中，prompt要求，输出需要执行的内容必须在```中，而且需要尽量保证每一个```中的内容是原子性的
            # 比如: 一个```json 中尽量不要存在两个json,后续会进行优化判定，当代码是不支持的
            new_line = line.replace('\n', '').strip()
            if len(new_line) > 3:
                content_type = str(new_line[3:])
            start_content = not start_content
        elif start_content:
            # python代码中的"""转义json会报错
            # python代码中的换行也会报错，都需要转义
            # line = line.replace("\r", '')
            if '"""' in line:
                line = line.replace('"""', '"')
                tri_quota = not tri_quota
            if "'" in line:
                line = line.replace('"', "'")
            if tri_quota:
                if '\r\n' in line:
                    line = line.replace("\r\n", '@@')
                elif '\n' in line:
                    line = line.replace('\n', '@@')
                elif '\r' in line:
                    line = line.replace("\r", '@@')
            line = line.replace("\r", '')
            # print(f'line:{line}')
            content += line
            print(f'content:{content}')
    if content != '':
        # 最后一个需要放到字典中去
        # content = ''.join(ccs)
        if content_type == 'json':
            contents[titles[title_idx-1]] = json.loads(content)
        else:
            contents[titles[title_idx-1]] = content
    return contents

contents = _parse_md(answer, titles=['用户需求', '验证项目'])
print(contents)

content:[
content:[    {
content:[    {        "desc": "创建项目目录并写入test.py文件",
content:[    {        "desc": "创建项目目录并写入test.py文件",        "create_dir": {
content:[    {        "desc": "创建项目目录并写入test.py文件",        "create_dir": {            "dir_path": "test",
content:[    {        "desc": "创建项目目录并写入test.py文件",        "create_dir": {            "dir_path": "test",            "file_path": "test/test.py",
content:[    {        "desc": "创建项目目录并写入test.py文件",        "create_dir": {            "dir_path": "test",            "file_path": "test/test.py",            "file_content": "def print_hello_world():@@
content:[    {        "desc": "创建项目目录并写入test.py文件",        "create_dir": {            "dir_path": "test",            "file_path": "test/test.py",            "file_content": "def print_hello_world():@@    print('Hello World')@@
content:[    {        "desc": "创建项目目录并写入test.py文件",        "create_dir": {            "dir_path": "test",            "file_path": "test/test.py",            "file_conte

data:{'case_id': 24723}
{'code': 0, 'message': '请求处理成功'}
data:{'case_id': 24724}
{'code': 0, 'message': '请求处理成功'}
data:{'case_id': 24725}
{'code': 0, 'message': '请求处理成功'}
data:{'case_id': 24726}
{'code': 0, 'message': '请求处理成功'}
data:{'case_id': 24727}
{'code': 0, 'message': '请求处理成功'}
data:{'case_id': 24728}
{'code': 0, 'message': '请求处理成功'}
data:{'case_id': 24729}
{'code': 0, 'message': '请求处理成功'}
data:{'case_id': 24730}
{'code': 0, 'message': '请求处理成功'}
data:{'case_id': 24731}
{'code': 0, 'message': '请求处理成功'}
data:{'case_id': 24732}
{'code': 0, 'message': '请求处理成功'}
data:{'case_id': 24733}
{'code': 0, 'message': '请求处理成功'}
data:{'case_id': 24734}
{'code': 0, 'message': '请求处理成功'}
data:{'case_id': 24735}
{'code': 0, 'message': '请求处理成功'}
data:{'case_id': 24736}
{'code': 0, 'message': '请求处理成功'}
data:{'case_id': 24737}
{'code': 0, 'message': '请求处理成功'}
data:{'case_id': 24738}
{'code': 0, 'message': '请求处理成功'}
data:{'case_id': 24739}
{'code': 0, 'message': '请求处理成功'}
data:{'case_id': 24740}
{'code'